# 4.3 알고리즘 선택하기

In [ ]:
# 공통 처리

# 불필요한 경고 메시지 무시
import warnings
warnings.filterwarnings('ignore')

# 라이브러리 임포트
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# 한글 글꼴 설정
import platform

if platform.system() == 'Windows':
    plt.rc('font', family='Malgun Gothic')
elif platform.system() == 'Darwin':
    plt.rc('font', family='Apple Gothic')    

# 데이터프레임 출력용 함수
from IPython.display import display

# 숫자 출력 조정
# 넘파이 부동소수점 출력 자리수 설정
np.set_printoptions(suppress=True, precision=4)

# 판다스 부동소수점 출력 자리수 설정
pd.options.display.float_format = '{:.4f}'.format

# 데이터프레임 모든 필드 출력
pd.set_option("display.max_columns",None)

# 그래프 글꼴 크기 설정
plt.rcParams["font.size"] = 14

# 난수 시드
random_seed = 123

### 4.3.2 예제 코드에서 사용할 데이터

In [ ]:
# 예제 데이터 생성하기

# 라이브러리 임포트
from sklearn.datasets import make_moons
from sklearn.datasets import make_circles
from sklearn.datasets import make_classification

# 선형 데이터
X1, y1 = make_classification(n_features=2, n_redundant=0,
    n_informative=2, random_state=random_seed,
    n_clusters_per_class=1, n_samples=200, n_classes=2)

# 초승달형 데이터 (선형 분리 불가능)
X2, y2 = make_moons(noise = 0.05, random_state=random_seed,
    n_samples=200)

# 원형 데이터 (선형 분리 불가능)
X3, y3 = make_circles(noise = 0.02, random_state=random_seed,
    n_samples=200)

# 세 가지 데이터를 DataList 변수에 저장
DataList = [(X1, y1), (X2, y2), (X3, y3)]

# N: 데이터의 가짓수
N = len(DataList)

In [ ]:
# 산포도 그리기
plt.figure(figsize=(15,4))

# 컬러맵 정의
from matplotlib.colors import ListedColormap
cmap = ListedColormap(['#0000FF', '#000000'])

for i, data in enumerate(DataList):
    X, y = data
    ax = plt.subplot(1, N, i+1)
    ax.scatter(X[:,0], X[:,1], c=y, cmap=cmap)

plt.show()

In [ ]:
from sklearn.model_selection import train_test_split

# 결정 경계를 그리는 함수
def plot_boundary(ax, x, y, algorithm):
    x_train, x_test, y_train, y_test = train_test_split(x, y,
            test_size=0.5, random_state=random_seed)
    # 컬러맵 정의
    from matplotlib.colors import ListedColormap
    cmap1 = plt.cm.bwr
    cmap2 = ListedColormap(['#0000FF', '#000000'])

    h = 0.005
    algorithm.fit(x_train, y_train)
    score_test = algorithm.score(x_test, y_test)
    score_train = algorithm.score(x_train, y_train)
    f1_min = x[:, 0].min() - 0.5
    f1_max = x[:, 0].max() + 0.5
    f2_min = x[:, 1].min() - 0.5
    f2_max = x[:, 1].max() + 0.5
    f1, f2 = np.meshgrid(np.arange(f1_min, f1_max, h), 
                         np.arange(f2_min, f2_max, h))
    if hasattr(algorithm, "decision_function"):
        Z = algorithm.decision_function(np.c_[f1.ravel(), f2.ravel()])
        Z = Z.reshape(f1.shape)
        ax.contour(f1, f2, Z, levels=[0], linewidth=2)
    else:
        Z = algorithm.predict_proba(np.c_[f1.ravel(), f2.ravel()])[:, 1]
        Z = Z.reshape(f1.shape)
    ax.contourf(f1, f2, Z, cmap=cmap1, alpha=0.3)
    ax.scatter(x_test[:,0], x_test[:,1], c=y_test, cmap=cmap2)
    ax.scatter(x_train[:,0], x_train[:,1], c=y_train, cmap=cmap2, marker='x')
    text = f'검증:{score_test:.2f}  학습: {score_train:.2f}'
    ax.text(f1.max() - 0.3, f2.min() + 0.3, text, horizontalalignment='right',
    fontsize=18) 
    

In [ ]:
# 산점도와 결정 경계를 그리는 함수

def plot_boundaries(algorithm, DataList):
    plt.figure(figsize=(15,4))
    for i, data in enumerate(DataList):
        X, y = data
        ax = plt.subplot(1, N, i+1)
        plot_boundary(ax, X, y, algorithm)
    plt.show()

### 4.3.3 로지스틱 회귀

#### 시그모이드 함수의 그래프

In [ ]:
# 시그모이드 함수의 정의
def sigmoid(x):
    return 1/(1 + np.exp(-x))

# x축 데이터 준비
x = np.linspace(-5, 5, 101)

# y축 데이터 준비
y = sigmoid(x)

# 그래프 작도
plt.plot(x, y, label='시그모이드 함수', c='b', lw=2)

# 범례 작도
plt.legend()

# 그리드 작도
plt.grid()

# 전체 그래프 출력
plt.show()

In [ ]:
# 로지스틱 회귀 알고리즘을 이용한 분류 결과와 산점도

# 알고리즘 정의
from sklearn.linear_model import LogisticRegression
algorithm = LogisticRegression(random_state=random_seed)

# 알고리즘의 파라미터 출력
# print(algorithm)
print(algorithm.get_params())


# 결과 표시 함수를 호출한다
plot_boundaries(algorithm, DataList)

### 4.3.4 서포트 벡터 머신(커널)

In [ ]:
# SVM(커널)을 이용한 분류 결과 및 산점도

# 알고리즘 선택하기
from sklearn.svm import SVC
algorithm = SVC(kernel='rbf', random_state=random_seed)

# 알고리즘에 설정된 파라미터 출력
# print(algorithm)
print(algorithm.get_params())

# 결과 표시 함수를 호출
plot_boundaries(algorithm, DataList)

### 4.3.5 신경망 알고리즘

In [ ]:
# 신경망을 이용한 분류 결과와 산점도

# 알고리즘 선택하기
from sklearn.neural_network import MLPClassifier
algorithm = MLPClassifier(random_state=random_seed)

# 알고리즘에 설정된 파라미터 출력
# print(algorithm)
print(algorithm.get_params())

# 결과 표시 함수를 호출
plot_boundaries(algorithm, DataList)

In [ ]:
# 신경망을 이용한 분류 결과와 산점도

# 알고리즘 선택하기
# 중간층 노드 수=(100,100)
from sklearn.neural_network import MLPClassifier
algorithm = MLPClassifier(hidden_layer_sizes=(100,100), random_state=random_seed)

# 알고리즘에 설정된 파라미터 출력
# print(algorithm)
print(algorithm.get_params())

# 결과 표시 함수를 호출
plot_boundaries(algorithm, DataList)

### 4.3.6 결정 트리

In [ ]:
# 추가 라이브러리 임포트
import seaborn as sns

# 예제 데이터 읽어 들이기
df_iris = sns.load_dataset("iris")

# 데이터 범위를 2종으로 축소
df2 = df_iris[50:150]

# 데이터 분리
X = df2.drop('species', axis=1)
y = df2['species']

In [ ]:
## 역주: 'GraphViz's executables not found' 오류 발생시에는
## https://graphviz.org/download/ 에서 환경에 맞는 버전의 Graphviz를 설치하고
## 아래 코드의 <설치경로>를 자신이 Graphviz를 설치한 경로로 수정한 후
## 진행하기 바랍니다.

import os
os.environ["PATH"] += os.pathsep + 'C:/Program Files/Graphviz/bin'
#os.environ["PATH"] += os.pathsep + '<설치경로>'

# 학습 
from sklearn.tree import DecisionTreeClassifier
algorithm = DecisionTreeClassifier(random_state=random_seed)
algorithm.fit(X, y)

# 결정 트리를 그래프 구조로 출력
from sklearn import tree
with open('iris-dtree.dot', mode='w') as f:
    tree.export_graphviz( algorithm, out_file=f,
        feature_names=X.columns, filled=True, rounded=True,
        special_characters=True, impurity=False, proportion=False
    )
import pydotplus
from IPython.display import Image
graph = pydotplus.graphviz.graph_from_dot_file('iris-dtree.dot')
graph.write_png('iris-dtree.png')
Image(graph.create_png())

In [ ]:
# 결정 트리를 이용한 분류 결과와 산점도

# 알고리즘 선택하기
from sklearn.tree import DecisionTreeClassifier
algorithm = DecisionTreeClassifier(random_state=random_seed)

# 알고리즘에 설정된 파라미터 출력
# print(algorithm)
print(algorithm.get_params())

# 결과 표시 함수를 호출
plot_boundaries(algorithm, DataList)

In [ ]:
# 결정 트리를 이용한 분류 결과와 산점도(max_depth=3으로 설정)

# 알고리즘 선택하기
from sklearn.tree import DecisionTreeClassifier
algorithm = DecisionTreeClassifier(max_depth=3,
    random_state=random_seed)

# 알고리즘에 설정된 파라미터 출력
# print(algorithm)
print(algorithm.get_params())

# 결과 표시 함수를 호출
plot_boundaries(algorithm, DataList)

### 4.3.7 랜덤 포레스트

In [ ]:
# 랜덤 포레스트를 이용한 분류 결과와 산점도

# 알고리즘 선택하기
from sklearn.ensemble import RandomForestClassifier
algorithm = RandomForestClassifier(random_state=random_seed)

# 알고리즘에 설정된 파라미터 출력
# print(algorithm)
print(algorithm.get_params())

# 결과 표시 함수를 호출
plot_boundaries(algorithm, DataList)

### 4.3.8 XGBoost

In [ ]:
# XGBoost를 이용한 분류 결과와 산점도

# 알고리즘 선택하기
import xgboost
algorithm = xgboost.XGBClassifier(random_state=random_seed)

# 알고리즘에 설정된 파라미터 출력
# print(algorithm)
print(algorithm.get_params())

# 결과 표시 함수를 호출
plot_boundaries(algorithm, DataList)

### 참고
버전 확인용 코드

In [ ]:
import sklearn
print(sklearn.__version__)